# Explore FVCOM GOM3 Hindcast on AWS Public Data

In [ ]:
import xarray as xr
import zarr

In [ ]:
url = 's3://umassd-fvcom/gom3/hindcast/parquet/combined.parq'    #  AWS Public Data

In [ ]:
to = dict(anon=True)
ro = dict(anon=True, asyncronous=True)

In [ ]:
if zarr.__version__[0]=='2':
    ds = xr.open_dataset(url, engine='kerchunk', chunks={'time':1}, 
                    backend_kwargs=dict(storage_options=dict(target_options=to,
                    remote_protocol='s3', lazy=True, remote_options=to)))
elif zarr.__version__[0]=='3':
    ds = xr.open_dataset(url, engine='kerchunk', chunks={'time':1}, 
                    backend_kwargs=dict(storage_options=dict(target_options=to,
                    remote_protocol='s3', lazy=True, remote_options=ro)))

In [ ]:
ds

## Visualize data on the triangular mesh

In [ ]:
import numpy as np
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import hvplot.xarray
import holoviews.operation.datashader as dshade
import pandas as pd

dshade.datashade.precompute = True
hv.extension('bokeh')

In [ ]:
var_name='temp'  # variable
level = 0  # vertical level (0 is surface, -1 is bottom)
time = '2015-11-01 04:00'  # time in UTC

In [ ]:
%%time
values = ds[var_name].isel(siglay=0).sel(time=time, method='nearest').load()

In [ ]:
v = np.vstack((ds['lon'], ds['lat'], values)).T
verts = pd.DataFrame(v, columns=['lon','lat','var'])

In [ ]:
points = gv.operation.project_points(gv.Points(verts, vdims=['var']))

In [ ]:
tris = pd.DataFrame(ds['nv'].T.values.astype('int')-1, columns=['v0','v1','v2'])

In [ ]:
tiles = gv.tile_sources.OSM

In [ ]:
title = f'{var_name}@level {level}   time:{time}'

In [ ]:
trimesh = gv.TriMesh((tris, points), label=title)
mesh = dshade.rasterize(trimesh).opts(
              cmap='rainbow', colorbar=True, width=600, height=400)

In [ ]:
tiles * mesh